FM(Factorization Machine)系列主要参考美团的[《深入FFM原理与实践》](https://tech.meituan.com/2016/03/03/deep-understanding-of-ffm-principles-and-practices.html)这篇博客内容做介绍并实现，首先介绍一下原理

### 一.FM原理
大家可能用过sklearn中的这个多项式特征处理函数：*`sklearn.preprocessing.PolynomialFeatures`*，它作用是就是将原始特征扩展为多项式特征，比如原始特征为$a,b$，那么它会做如下扩展：   

$$
[a,b]\rightarrow [1,a,b,a^2,ab,b^2]
$$  

而FM的初衷便是对这组新特征做线性建模，一般地，它可以表示如下：   

$$
y(x)=w_0+\sum_{i=1}^nw_ix_i+\sum_{i=1}^{n-1}\sum_{j=i+1}^nw_{ij}x_ix_j
$$  

FM通常不会对平方项建模（比如上面的$a^2,b^2$），这里$n$代表样本的特征数量，$x_i$是第$i$个特征值，$w_0,w_i,w_{ij}$是模型参数，到这里大家可能会有疑惑，我们干嘛不先通过多项式特征处理函数做转换，然后再接着做一个线性回归或者logistic回归之类的不就行了吗？那这个...FM拿它何用？如果按照刚刚的操作其实是有很大问题的，主要有两点问题：    

（1）**参数爆炸**；   

（2）**高维稀疏**；  

第一点比较容易理解，对于$n$个特征，$w_{ij}$将会有$\frac{n(n-1)}{2}$项，形象点说就是平常随意用到的100个特征，扩展后将会有5000个，而参数越多，如果没有足够的数据做训练，模型很容易陷入过拟合，而对于第二点，经常处理离散特征的同学会很容易理解，比如下图
![avatar](./source/17_FM_one-hot1.png)  

包含有三个特征（最左侧的是标签），且都是离散特征，而对于离散特征我们经常做的一种操作便是one-hot转换，转换后的结果如下图：   

![avatar](./source/17_FM_one-hot2.png)  

如果我们在对这些特征做多项式转换，可以发现转后的20多个特征中，仅仅只有3个非零特征，这就意味着绝大部分的$x_ix_j$将会是0，而损失函数关于$w_{ij}$的导数必然会包含有$x_ix_j$这一项，这就意味$w_{ij}$大部分情况下就是个摆设，很难被更新到，而FM便可以解决这两个问题，它假设$w_{ij}$由两个向量的内积生成：   

$$
w_{ij}:=<v_i,v_j>
$$  

这里，$v_i$表示第$i$个特征的隐向量，其向量长度为$k(k<<n)$，通常$k=4$即可，这时FM模型方程如下：   

$$
y(x)=w_0+\sum_{i=1}^nw_ix_i+\sum_{i=1}^{n-1}\sum_{j=i+1}^n<v_i,v_j>x_ix_j
$$   
进一步的，我们可以将其化简为如下形式：   

$$
y(x)=w_0+\sum_{i=1}^nw_ix_i+\frac{1}{2}\sum_{f=1}^k((\sum_{i=1}^nv_{i,f}x_i)^2-\sum_{i=1}^nv_{i,f}^2x_i^2)
$$  

这里，$v_{i,f}$表示向量$v_i$的第$f$个元素，上述化简用到了这样的关系：$ab+ac+bc=\frac{1}{2}((a+b+c)^2-(a^2+b^2+c^2))$，接下来我们可以进一步看看梯度：   

$$
\frac{\partial}{\partial\theta}y(x)=\left\{\begin{matrix}
1 &\theta=w_0 \\ 
x_i &\theta=w_i \\ 
x_i\sum_{j=1}^nv_{j,f}x_j-v_{i,f}x_i^2 & \theta=v_{i,f}
\end{matrix}\right.
$$  

可以发现前面的两个问题可以被FM解决了，第一个问题，参数量从$n(n-1)/2$降低到了$kn$，第二个高维稀疏导致参数无法被训练的问题，对于$v_{i,f}$只要$x_i$不为0，且所有$x_j,j=1,2,...,n$中有一个不为0，那么梯度$\frac{\partial}{\partial v_{i,f}}y(x)$就不为0，这比$x_ix_j$不为0的条件松了很多

### 二.代码实现
这里就对FM应用到回归任务做简单实现，更多的功能扩展放到FFM中，下面推导一下损失函数对参数的梯度，假设样本$x$对应的标签为$t$，那么损失函数可以表示如下：   

$$
L(\theta)=\frac{1}{2}(y(x)-t)^2
$$   

那么：   

$$
\frac{\partial L(\theta)}{\partial y(x)}=y(x)-t
$$   

再根据链式求导有：   

$$
\frac{\partial L(\theta)}{\partial \theta}=\frac{\partial L(\theta)}{\partial y(x)}\frac{\partial y(x)}{\partial\theta}\\
=(y(x)-t)\cdot \left\{\begin{matrix}
1 &\theta=w_0 \\ 
x_i &\theta=w_i \\ 
x_i\sum_{j=1}^nv_{j,f}x_j-v_{i,f}x_i^2 & \theta=v_{i,f}
\end{matrix}\right.
$$

In [1]:
"""
FM因子分解机的简单实现，只实现了损失函数为平方损失的回归任务，更多功能扩展请使用后续的FFM
代码封装到ml_models.fm中
"""
import numpy as np


class FM(object):
    def __init__(self, epochs=1, lr=1e-3, batch_size=2, hidden_dim=4, lamb=1e-3, normal=True):
        """

        :param epochs: 迭代轮数
        :param lr: 学习率
        :param batch_size:
        :param hidden_dim:隐变量维度
        :param lamb:l2正则化系数
        :param normal:是否归一化，默认用min-max归一化
        """
        self.epochs = epochs
        self.lr = lr
        self.batch_size = batch_size
        self.hidden_dim = hidden_dim
        self.lamb = lamb
        # 初始化参数
        self.w = None  # w_0,w_i
        self.V = None  # v_{i,f}
        # 归一化
        self.normal = normal
        if normal:
            self.xmin = None
            self.xmax = None

    def _y(self, X):
        """
        实现y(x)的功能
        :param X:
        :return:
        """
        # 去掉第一列bias
        X_ = X[:, 1:]
        n_sample, n_feature = X_.shape
        pol = (X_.reshape((n_sample, n_feature, 1)) * X_.reshape((n_sample, 1, n_feature))) * (
            1 - np.eye(n_feature)) * (
                  self.V @ self.V.T)
        return X @ self.w.reshape(-1) + 0.5 * np.sum(pol, axis=(1, 2))

    def fit(self, X, y):
        if self.normal:
            self.xmin = X.min(axis=0)
            self.xmax = X.max(axis=0)
            X = (X - self.xmin) / self.xmax
        # 记录loss
        losses = []
        n_sample, n_feature = X.shape
        x_y = np.c_[np.ones(n_sample), X, y]
        # 初始化参数
        self.w = np.random.random((n_feature + 1, 1))
        self.V = np.random.random((n_feature, self.hidden_dim))
        # 更新参数
        count = 0
        for _ in range(self.epochs):
            np.random.shuffle(x_y)
            for index in range(x_y.shape[0] // self.batch_size):
                count += 1
                batch_x_y = x_y[self.batch_size * index:self.batch_size * (index + 1)]
                batch_x = batch_x_y[:, :-1]
                batch_y = batch_x_y[:, -1:]
                # 计算y(x)-t
                y_x_t = self._y(batch_x).reshape((-1, 1)) - batch_y
                # 更新w
                self.w = self.w - (self.lr * (np.sum(y_x_t * batch_x, axis=0) / self.batch_size).reshape(
                    (-1, 1)) + self.lamb * self.w)
                # 更新 V
                batch_x_ = batch_x[:, 1:]
                V_X = batch_x_ @ self.V
                X_V_X = batch_x_.reshape((batch_x_.shape[0], batch_x_.shape[1], 1)) * V_X.reshape(
                    (V_X.shape[0], 1, V_X.shape[1]))
                X_2 = batch_x_ * batch_x_
                V_X_2 = X_2.reshape((X_2.shape[0], X_2.shape[1], 1)) * self.V.reshape(
                    (1, self.V.shape[0], self.V.shape[1]))
                self.V = self.V - self.lr * (np.sum(y_x_t * (X_V_X - V_X_2),
                                                    axis=0) / self.batch_size + self.lamb * self.V)
                # 计算loss
                loss = np.sum(np.abs(y - self.predict(X))) / n_sample
                losses.append(loss)
        return losses

    def predict(self, X):
        """
        :param X:
        :return:
        """
        if self.normal:
            X = (X - self.xmin) / self.xmax
        n_sample, n_feature = X.shape
        pol = (X.reshape((n_sample, n_feature, 1)) * X.reshape((n_sample, 1, n_feature))) * (
            1 - np.eye(n_feature)) * (
                  self.V @ self.V.T)
        return np.c_[np.ones(n_sample), X] @ self.w.reshape(-1) + 0.5 * np.sum(pol, axis=(1, 2))

### 三.测试

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#造伪数据
data1 = np.linspace(1, 10, num=100)
data2 = np.linspace(1, 10, num=100) + np.random.random(size=100)
target = data1 * 2 + data2 * 1 + 10 * data1 * data2 + np.random.random(size=100)
data = np.c_[data1, data2]

In [4]:
#训练模型
model = FM(batch_size=1, lr=1e-3, epochs=1, lamb=1e-3, hidden_dim=2)
losses = model.fit(data, target)

In [5]:
#查看拟合效果
plt.scatter(data[:, 0], target)
plt.plot(data[:, 0], model.predict(data), color='r')

In [6]:
#查看损失值变化
plt.plot(range(0, len(losses)), losses)